In [46]:
# import libraries
import pandas as pd
import numpy as np
import scipy as sp

# Qestion 1
# 1.a
# import data in the us_shipping directory to a Pandas dataframe
raw_data = "./us_shipping/us_shipping.csv"
df = pd.DataFrame.from_csv(raw_data,header=0, index_col=None)


#replace missing values
#replace X and S with nan
for col in df.columns[3:7]:
    df[col] = df[col].replace("S",np.nan)
    df[col] = df[col].replace("X",np.nan)
    df[col] = df[col].replace("Z",np.nan)

In [148]:
#Question 1.b Create a new dataframe

# firstly get all state names in the file
states = set(df["Origin"])
states = sorted(states)

# now get all modes in the file
modes = set(df["Mode"])
modes = sorted(modes)

# print states
# print modes

#create a new data frame object to hold data
new_df_2012_value = pd.DataFrame(0, index=states,columns=modes)
new_df_2007_value = pd.DataFrame(0, index=states,columns=modes)
modes

['Air (incl truck and air)',
 'All modes',
 'Deep sea',
 'For-hire truck',
 'Great Lakes',
 'Inland water',
 'Multiple Waterways',
 'Multiple modes',
 'Other modes',
 'Other multiple modes',
 'Parcel, U.S.P.S. or courier',
 'Pipeline',
 'Private truck',
 'Rail',
 'Rail and water',
 'Single modes',
 'Truck',
 'Truck and rail',
 'Truck and water',
 'Water']

In [48]:
# fill the new data frame by iterating the original df 
# new_df_2012_value is a dataframe that has export information in terms of value in year 2012
# new_df_2017_value is a dataframe that has export information in terms of value in year 2017
# referred to http://pandas.pydata.org/pandas-docs/stable/indexing.html

for index, row in df.iterrows():
    if row["Year"]==2012:
        new_df_2012_value.loc[row["Origin"],row["Mode"]] = row["Value"]
        

for index, row in df.iterrows():
    if row["Year"]==2007:
        new_df_2007_value.loc[row["Origin"],row["Mode"]] = row["Value"]
        

In [16]:
# Question 1.d
# create a function that returns a dictionary that stores proportional value of exports shipped by a given state
def proportion(dataFrame, state):
    row = dataFrame.loc[state]
    ret={}
    for mode in modes:
        if row[mode] >0 :
            ret[mode] = float(row[mode])/float(row["All modes"])
    return ret
            
dat = proportion(new_df_2012_value,"Tennessee")


print dat["For-hire truck"] +dat["Private truck"]
print dat["Truck"]

0.780676322636
0.780676322636


In [56]:
# Question 1.e
# find the modes had the biggest positive and negative differences in tons-shipped from 2007 to 2012

# firstly we can construct a dataframe to store all shippment in tons by modes in 2007 and 2012
df_shipped_tones = pd.DataFrame(0, index={2007,2012},columns=modes)

# now fill the dataframe by going through the orignal df  
for index, row in df.iterrows():
    if row["Year"]==2007 and row["Tons"]>0:
        df_shipped_tones.loc[2007,row["Mode"]] += float(row["Tons"])
    elif row["Year"]==2012 and row["Tons"]>0:
        df_shipped_tones.loc[2012,row["Mode"]] += float(row["Tons"])


df_shipped_tones


,Air (incl truck and air),All modes,Deep sea,For-hire truck,Great Lakes,Inland water,Multiple Waterways,Multiple modes,Other modes,Other multiple modes,"Parcel, U.S.P.S. or courier",Pipeline,Private truck,Rail,Rail and water,Single modes,Truck,Truck and rail,Truck and water,Water
2012,4205.0,11299406.0,60302.0,4298693.0,31403.0,356752.0,36926.0,336511.0,30831.0,2418.0,28314.0,593510.0,3761473.0,1609268.0,35947.0,10905516.0,8060169.0,195262.0,21195.0,514445.0
2007,2575.0,12543426.0,36370.0,4074037.0,16215.0,306945.0,0.0,550045.0,199405.0,89505.0,33899.0,621762.0,4626270.0,1839268.0,22591.0,11698131.0,8778710.0,187732.0,119666.0,364413.0


In [57]:
# add a row "Difference" in df df_shipped_tones to store difference in tons by modes
for mode in modes:
    tons_2012 = df_shipped_tones.loc[2012,mode]
    tons_2007 = df_shipped_tones.loc[2007,mode]
    df_shipped_tones.loc["Difference",mode] = tons_2012-tons_2007
    
df_shipped_tones

,Air (incl truck and air),All modes,Deep sea,For-hire truck,Great Lakes,Inland water,Multiple Waterways,Multiple modes,Other modes,Other multiple modes,"Parcel, U.S.P.S. or courier",Pipeline,Private truck,Rail,Rail and water,Single modes,Truck,Truck and rail,Truck and water,Water
2012,4205.0,11299406.0,60302.0,4298693.0,31403.0,356752.0,36926.0,336511.0,30831.0,2418.0,28314.0,593510.0,3761473.0,1609268.0,35947.0,10905516.0,8060169.0,195262.0,21195.0,514445.0
2007,2575.0,12543426.0,36370.0,4074037.0,16215.0,306945.0,0.0,550045.0,199405.0,89505.0,33899.0,621762.0,4626270.0,1839268.0,22591.0,11698131.0,8778710.0,187732.0,119666.0,364413.0
Difference,1630.0,-1244020.0,23932.0,224656.0,15188.0,49807.0,36926.0,-213534.0,-168574.0,-87087.0,-5585.0,-28252.0,-864797.0,-230000.0,13356.0,-792615.0,-718541.0,7530.0,-98471.0,150032.0


In [69]:
ton_difference = df_shipped_tones.loc["Difference"]
max_in_tone=max(abs(i) for i in ton_difference)
max_in_tone


1244020.0

In [70]:
# Question 1.e
# find the state decreased the most in the export value across all modes from 2007 to 2012

# firstly we can construct a dataframe to store all shippment in value by modes in 2007 and 2012
df_shipped_value = pd.DataFrame(0, index={2007,2012},columns=modes)

# now fill the dataframe by going through the orignal df  
for index, row in df.iterrows():
    if row["Year"]==2007 and row["Value"]>0:
        df_shipped_value.loc[2007,row["Mode"]] += float(row["Value"])
    elif row["Year"]==2012 and row["Value"]>0:
        df_shipped_value.loc[2012,row["Mode"]] += float(row["Value"])


df_shipped_value

,Air (incl truck and air),All modes,Deep sea,For-hire truck,Great Lakes,Inland water,Multiple Waterways,Multiple modes,Other modes,Other multiple modes,"Parcel, U.S.P.S. or courier",Pipeline,Private truck,Rail,Rail and water,Single modes,Truck,Truck and rail,Truck and water,Water
2012,419575.0,13852141.0,23092.0,6504637.0,425.0,213031.0,19169.0,1950753.0,706.0,328.0,1683132.0,489844.0,3627591.0,445568.0,4064.0,11900363.0,10132228.0,222158.0,19567.0,295655.0
2007,225352.0,11684872.0,16413.0,4955475.0,241.0,79862.0,0.0,1866722.0,257619.0,38330.0,1560226.0,384624.0,3380092.0,387355.0,8072.0,9539036.0,8335789.0,184108.0,40922.0,104065.0


In [72]:
# add a row "Difference" in df df_shipped_value to store difference in value by modes
for mode in modes:
    value_2012 = df_shipped_value.loc[2012,mode]
    value_2007 = df_shipped_value.loc[2007,mode]
    df_shipped_value.loc["Difference",mode] = value_2012-value_2007
    
df_shipped_value

,Air (incl truck and air),All modes,Deep sea,For-hire truck,Great Lakes,Inland water,Multiple Waterways,Multiple modes,Other modes,Other multiple modes,"Parcel, U.S.P.S. or courier",Pipeline,Private truck,Rail,Rail and water,Single modes,Truck,Truck and rail,Truck and water,Water
2012,419575.0,13852141.0,23092.0,6504637.0,425.0,213031.0,19169.0,1950753.0,706.0,328.0,1683132.0,489844.0,3627591.0,445568.0,4064.0,11900363.0,10132228.0,222158.0,19567.0,295655.0
2007,225352.0,11684872.0,16413.0,4955475.0,241.0,79862.0,0.0,1866722.0,257619.0,38330.0,1560226.0,384624.0,3380092.0,387355.0,8072.0,9539036.0,8335789.0,184108.0,40922.0,104065.0
Difference,194223.0,2167269.0,6679.0,1549162.0,184.0,133169.0,19169.0,84031.0,-256913.0,-38002.0,122906.0,105220.0,247499.0,58213.0,-4008.0,2361327.0,1796439.0,38050.0,-21355.0,191590.0


In [147]:
ton_difference = df_shipped_value.loc["Difference"]
decreased_most=min(i for i in ton_difference)
decreased_most
ton_difference

Air (incl truck and air)        194223.0
All modes                      2167269.0
Deep sea                          6679.0
For-hire truck                 1549162.0
Great Lakes                        184.0
Inland water                    133169.0
Multiple Waterways               19169.0
Multiple modes                   84031.0
Other modes                    -256913.0
Other multiple modes            -38002.0
Parcel, U.S.P.S. or courier     122906.0
Pipeline                        105220.0
Private truck                   247499.0
Rail                             58213.0
Rail and water                   -4008.0
Single modes                   2361327.0
Truck                          1796439.0
Truck and rail                   38050.0
Truck and water                 -21355.0
Water                           191590.0
Name: Difference, dtype: float64